In [6]:
from typing import Tuple

from trials.ModelSpec import SpecWrapper
from trials.Constants import nasbench, ALL_HASH
from trials.Utilities import get_spec
from trials.Utilities import random_spec
from typing import List
from typing import Union
import os
import json
from matplotlib import pyplot as plt
import numpy as np
from IPython.display import clear_output
from scipy import signal

In [7]:
def clear_print(text: str, wait: bool = True) -> None:
    clear_output(wait=wait)
    print(text)

In [8]:
PAD_LENGTH = 15
FIG_SIZE = (16, 8)

In [9]:
def smoothed_line_figure(y_half: Union[List[float], np.ndarray], y_full: Union[List[float], np.ndarray]) -> Tuple[plt.Figure, plt.Axes]:
    assert len(y_half) == len(y_full)

    x = np.arange(len(y_half))
    filter_b, filter_a = signal.butter(8, 0.05)

    y_half_smoothed = signal.filtfilt(filter_b, filter_a, y_half, padlen=PAD_LENGTH)
    y_full_smoothed = signal.filtfilt(filter_b, filter_a, y_full, padlen=PAD_LENGTH)

    fig, axis = plt.subplots(figsize=FIG_SIZE)

    lines_y_half_smoothed = plt.plot(x, y_half_smoothed, color='black', alpha=1.0)
    lines_y_half = plt.plot(x, y_half, color='black', alpha=0.25, marker='o')

    lines_y_full_smoothed = plt.plot(x, y_full_smoothed, color='orange', alpha=1.0)
    lines_y_full = plt.plot(x, y_full, color='orange', alpha=0.25, marker='o')

    return (fig, axis)

In [12]:
graphs_path = os.path.join(os.getcwd(), 'graphs')
for trial_name in os.listdir(graphs_path):
    is_half = "Half" in trial_name
    trial_path = os.path.join(graphs_path, trial_name)

    if not os.path.isdir(trial_path):
        continue

    for epoch_name in os.listdir(trial_path):
        y_half = []
        y_full = []
        std_dv_half = []
        std_dv_full = []

        title_accuracy = f'{trial_name} - {epoch_name} - Median Total Accuracy'
        path_accuracy = os.path.join(os.path.join(graphs_path, f'{title_accuracy}.png'))
        title_delta = f'{trial_name} - {epoch_name} - Median Total Accuracy Delta'
        path_delta = os.path.join(os.path.join(graphs_path, f'{title_accuracy}.png'))

        # if os.path.exists(path_accuracy) and os.path.exists(path_delta):
        #     continue

        heredity_path = os.path.join(trial_path, epoch_name, 'Heredity')
        for file_name in os.listdir(heredity_path):
            file_path = os.path.join(heredity_path, file_name)

            hashes = json.load(open(file_path, 'r', encoding='utf8'))
            specs_half = [get_spec(hsh, stop_halfway=False) for hsh in hashes]
            specs_full = [get_spec(hsh, stop_halfway=True) for hsh in hashes]

            scores_half = [ind.get_data().total_accuracy for ind in specs_half]
            y_half.append(np.median(scores_half))
            std_dv_half.append(np.std(scores_half))

            scores_full = [ind.get_data().total_accuracy for ind in specs_full]
            y_full.append(np.median(scores_full))
            std_dv_full.append(np.std(scores_full))

        fig, axis = smoothed_line_figure(y_half, y_full)
        plt.title(title_accuracy)
        plt.ylabel('Median Total Accuracy')
        plt.xlabel('Generation')
        plt.savefig(os.path.join(graphs_path, f'{title_accuracy}.png'))
        plt.close(fig)

        fig, axis = smoothed_line_figure(np.diff(y_half), np.diff(y_full))
        plt.title('Baseline Random - Training Velocity')
        plt.ylabel('Improvement Over Previous Generation')
        plt.xlabel('Generation')
        plt.savefig(os.path.join(graphs_path, f'{title_delta}.png'))
        plt.close(fig)



KeyboardInterrupt: 

In [ ]:
#trial_name = 'Baseline Random Half'
#epoch_name = 'Epoch 1'
#graphs_path = os.path.join(os.getcwd(), 'graphs')
#trial_path = os.path.join(graphs_path, trial_name)
#
#y_half = []
#y_full = []
#std_dv_half = []
#std_dv_full = []
#
#title_accuracy = f'{trial_name} - {epoch_name} - Median Total Accuracy'
#path_accuracy = os.path.join(os.path.join(graphs_path, f'{title_accuracy}.png'))
#title_delta = f'{trial_name} - {epoch_name} - Median Total Accuracy Delta'
#path_delta = os.path.join(os.path.join(graphs_path, f'{title_accuracy}.png'))
#
#heredity_path = os.path.join(trial_path, epoch_name, 'Heredity')
#for file_name in os.listdir(heredity_path):
#    file_path = os.path.join(heredity_path, file_name)
#
#    hashes = json.load(open(file_path, 'r', encoding='utf8'))
#    specs_half = [get_spec(hsh, stop_halfway=False) for hsh in hashes]
#    specs_full = [get_spec(hsh, stop_halfway=True) for hsh in hashes]
#
#    scores_half = [ind.get_data().total_accuracy for ind in specs_half]
#    y_half.append(np.median(scores_half))
#    std_dv_half.append(np.std(scores_half))
#
#    scores_full = [ind.get_data().total_accuracy for ind in specs_full]
#    y_full.append(np.median(scores_full))
#    std_dv_full.append(np.std(scores_full))

In [ ]:
#fig, ax = smoothed_line_figure(y_half, y_full)
#plt.title(title_accuracy)
#plt.ylabel('Median Total Accuracy')
#plt.xlabel('Generation')
#max_half = np.max(y_half)
#max_half_idx = np.argmax(y_half)
#ax.annotate(f'{max_half:0.2f}', (max_half_idx, max_half), textcoords='data')
## plt.savefig(os.path.join(graphs_path, f'{title_accuracy}.png'))
#plt.show()
#plt.close(fig)
#
#fig, ax = smoothed_line_figure(np.diff(y_half), np.diff(y_full))
#plt.title('Baseline Random - Training Velocity')
#plt.ylabel('Improvement Over Previous Generation')
#plt.xlabel('Generation')
#max_half = np.max(y_full)
#max_half_idx = np.argmax(y_half)
#ax.annotate(f'{max_half:0.2f}', (max_half_idx, max_half), textcoords='data')
## plt.savefig(os.path.join(graphs_path, f'{title_delta}.png'))
#plt.show()
#plt.close(fig)